In [3]:
!pip install -U ortools pandas

Requirement already up-to-date: ortools in /home/maa/miniconda3/envs/pytorch/lib/python3.7/site-packages (7.2.6977)
     |████████████████████████████████| 10.1MB 7.4MB/s 


In [4]:
import numpy as np
import pandas as pd
from ortools.linear_solver import pywraplp

In [18]:
np.random.seed(42)
samples = 500

In [46]:
index = ['name', 'cost', 'size', 'handling', 'positioning', 'joining']
names = ['robot' + str(i) for i in range(samples)]
costs = np.random.randint(1000, 10000, size=(samples)).tolist()
handling = np.random.binomial(1, 0.5, samples)
positioning = np.random.binomial(1, 0.5, samples)
joining = np.random.binomial(1, 0.5, samples)
sizes = np.round(np.random.rand((samples)) * 10, 2)

In [47]:
df = pd.DataFrame({'name': names, 'cost': costs, 'size': sizes, 'handling': handling, 'positioning': positioning, 'joining': joining})
df.head(10)

,name,cost,size,handling,positioning,joining
0,robot0,1785,9.37,0,1,0
1,robot1,7139,9.84,1,1,1
2,robot2,7344,7.96,0,1,0
3,robot3,3416,7.57,1,1,0
4,robot4,9988,6.81,1,1,1
5,robot5,5154,1.66,1,1,1
6,robot6,6619,0.59,0,1,0
7,robot7,2936,6.91,0,1,0
8,robot8,4807,7.29,1,1,1
9,robot9,8151,0.47,0,0,0


In [38]:
constraints = {'cost': 20000, 'size': 10.0, 'max_num': 5, 'handling': 8, 'positioning': 5, 'joining': 3}

In [40]:
solver = pywraplp.Solver('simple_lp_programm', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)
solver

<ortools.linear_solver.pywraplp.Solver; proxy of <Swig Object of type 'operations_research::MPSolver *' at 0x7f6cd5597ed0> >

### Create Variables
for every robot in the `df` set one variable which corresponds to the number of occurences for this robot. At first every variable is 1, then it will be minimized.
[example](https://developers.google.com/optimization/mip/integer_opt)

In [48]:
solver.Clear()
# fitness = solver.NumVar(0, solver.infinity(), 'fitness')
sol = [[]] * len(df)
objective_num = solver.Objective()
for i in range(0, len(df)):
    sol[i] = solver.IntVar(0, solver.infinity(), df['name'][i])
    objective_num.SetCoefficient(sol[i], 1)
objective_num.SetMinimization()
# cost = solver.NumVar(0, constraints['max_cost'], 'cost')
# size = solver.NumVar(0, constraints['max_size'], 'size')
# num = solver.IntVar(0, constraints['max_num'], 'num')

print('Number of variables:', solver.NumVariables())

Number of variables: 500


### create constraints
necessary:

cost $c$, size $s$ and num $n$


\begin{equation}
0 \leq c \leq c_{max} \qquad \in [0..20000] \\
0 \leq s \leq s_{max} \qquad \in [0..10] \\
0 \leq n \leq n_{max} \qquad \in \{ 0..5 \}
\end{equation}

and the number of handling, positionig and joining resources must be:

\begin{equation}
h_{min} \leq h \\
p_{min} \leq p \\
j_{min} \leq j
\end{equation}

In [49]:
# create constraints
# cost_c = solver.Constraint(0, constraints['max_cost'], 'cost')
# size_c = solver.Constraint(0, constraints['max_size'], 'size')
# num_c = solver.Constraint(0, constraints['max_num'], 'num')

# cost_c.SetCoefficient(cost, 1)
# size_c.SetCoefficient(size, 1)
# num_c.SetCoefficient(num, 1)
names_min = ['handling', 'positioning', 'joining']
names_max = ['cost', 'size']

const = [0] * (len(names_max) + len(names_min))
for i in range(len(names_max)):
    const[i] = solver.Constraint(0, constraints[names_max[i]], names_max[i])
    for j in range(len(df)):
        const[i].SetCoefficient(sol[j], float(df[names_max[i]][j]))
        

for i in range(len(names_min)):
    const[i] = solver.Constraint(constraints[names_min[i]], solver.infinity(), names_min[i])
    for j in range(len(df)):
        const[i].SetCoefficient(sol[j], float(df[names_min[i]][j]))

print('Number of constraints:', solver.NumConstraints())

Number of constraints: 5


## fitness function
\begin{equation}
f = \sum_{r \in \mathcal{R}} \frac{c_r}{C_m} + \frac{s_r}{S_m} \\
\text{with:} \\
\text{f = fitness} \\
\text{c = cost} \\
\text{s = size}
\end{equation}

In [36]:
def fitness(sol):
    f = sum(sol['cost'] / constraints['max_cost'] + sol['size'] / constraints['max_size'])
    return f

In [40]:
# create objective function, minimize fitness
# objective_fitness = solver.Objective()
# objective_fitness.SetCoefficient(const[0], 
# objective_fitness.SetMinimization()

In [62]:
solver.Solve()
print('Solution:')
print('Number of robots:', objective_num.Value())

cost = 0
size = 0
handling = 0
positioning = 0
joining = 0
for i in range(len(sol)):
    if sol[i].solution_value() > 0: 
        c = df.loc[df.name==str(sol[i]), 'cost'].item()
        s = df.loc[df.name==str(sol[i]), 'size'].item()
        handling += df.loc[df.name==str(sol[i]), 'handling'].item() * sol[i].solution_value()
        positioning += df.loc[df.name==str(sol[i]), 'positioning'].item() * sol[i].solution_value()
        joining += df.loc[df.name==str(sol[i]), 'joining'].item() * sol[i].solution_value()
        print(sol[i], '\tnum:', sol[i].solution_value(), '\tcost:', c, '€', '\tsize:', s, 'm2' )
        cost += c
        size += s

print('Cost =', cost, '€')
print('Size =', round(size, 3), 'm2')
print('Handling =', handling, '/', constraints['handling'])
print('Positioning =', positioning, '/', constraints['positioning'])
print('Joining =', joining, '/', constraints['joining'])

Solution:
Number of robots: 8.0
robot21 	num: 3.0 	cost: 1382 € 	size: 0.22 m2
robot169 	num: 5.0 	cost: 1436 € 	size: 0.01 m2
Cost = 2818 €
Size = 0.23 m2
Handling = 8.0 / 8
Positioning = 5.0 / 5
Joining = 3.0 / 3


In [52]:
df.loc[df.name.isin(['robot21', 'robot169']), :]

,name,cost,size,handling,positioning,joining
21,robot21,1382,0.22,1,0,1
169,robot169,1436,0.01,1,1,0
